### ファイル解析の雛形ノート


まず、ファイルを読み込み、中身を確認する。

In [1]:
targetFileName = "raw/LDP2017.html"

f = open(targetFileName, "r")
print( f.read() )

<section class="speech__list">
	<div class="speech__accordion">
		<div class="speech__list--a speech__accordion--btn">
			<dl>
				<dt>
					<img src="/election/results/sen_shu48/candidate/img/members/abe-shi.jpg" alt="安倍 晋三">
				</dt>
				<dd>
					<span class="kana">あべ しんぞう</span><br><strong class="name">安倍 晋三</strong><strong class="post">総裁</strong>
				</dd>
				<dd class="speech__list--btn">
					<img src="/election/results/sen_shu48/common/img/speech/img.gif" alt="">
				</dd>
			</dl>
		</div>
		<div class="speech__accordion--wrap">
			<div class="speech__accordion--inr">
				<h3 class="speech__list--title"><span>10月10日 ( 火 )</span></h3>
				<div class="speech__table">
					<div class="speech__table--list">
						<dl>
							<dt class="speech__table--head">
								<ul>
									<li class="time">開始時間</li>
									<li class="candidate">候補者</li>
									<li class="place">演説場所</li>
									<li class="janitorial">用務</li>
								</ul>
							</dt>
							<dd class="speech__table

HTML parserは、HTMLの構造を分析するライブラリ。タグの構造を解析して、わかりやすいように表示する。

In [2]:
from bs4 import BeautifulSoup  # 超有名なhtmlパーサーを使う宣言
import html  # htmlモジュールのunescapeは文字化けさせないおまじない

f = open(targetFileName, "r")
bsObj = BeautifulSoup( f.read(), "html")
print( bsObj.prettify() )  # prettifyは枝構造が明瞭になるように出力する

<section class="speech__list">
 <div class="speech__accordion">
  <div class="speech__list--a speech__accordion--btn">
   <dl>
    <dt>
     <img alt="安倍 晋三" src="/election/results/sen_shu48/candidate/img/members/abe-shi.jpg"/>
    </dt>
    <dd>
     <span class="kana">
      あべ しんぞう
     </span>
     <br/>
     <strong class="name">
      安倍 晋三
     </strong>
     <strong class="post">
      総裁
     </strong>
    </dd>
    <dd class="speech__list--btn">
     <img alt="" src="/election/results/sen_shu48/common/img/speech/img.gif"/>
    </dd>
   </dl>
  </div>
  <div class="speech__accordion--wrap">
   <div class="speech__accordion--inr">
    <h3 class="speech__list--title">
     <span>
      10月10日 ( 火 )
     </span>
    </h3>
    <div class="speech__table">
     <div class="speech__table--list">
      <dl>
       <dt class="speech__table--head">
        <ul>
         <li class="time">
          開始時間
         </li>
         <li class="candidate">
          候補者
         </li>
         

データの規則性を検討する。ここが知恵の見せ所。

BeautifulSoupでは、find_allで要件を満たすタグを列挙することができる。

ここでは、このテーブルに関する以下の分析結果を利用することとする。

1. speech__accordionクラスのdivタグが人を表す
2. speech__list--aクラスのdivのdl要素のdd要素の、nameクラスのstrong要素が、人名を表す
3. speech__accordion--inrクラスのdiv要素がスケジュールを表す
4. その中で、speech__list--titleクラスのh3要素（日付）と、speech__tableクラスのdivが交互に現れる。
5. speech__tableクラスのdivの中に、speech__table--memberクラスのdd要素がその日の個別会場を表す

In [3]:
f = open(targetFileName,"r")
bsObj = BeautifulSoup( f.read(), "html")

for person_div in bsObj.find_all("div", class_="speech__accordion"):   # divのspeech__accordionクラスを列挙
    # findを繋げて、タグの内側の要素を絞り込んでいく
    name = person_div.find("div", class_="speech__list--a").find("dl").find("dd").find("strong",class_="name").get_text()
    # print( name )
    schedules = person_div.find("div", class_="speech__accordion--inr")
    # 次はやや上級。zipは、二つ同時に列挙する関数。
    # ここでは「日付」と「演説会場のグループ」を同時に列挙する、
    for date, speeches in zip( schedules.find("h3", class_="speech__list--title"), schedules.find_all("div", class_="speech__table") ):
        # print(speeches)
        for speech in speeches.find("dd",class_="speech__table--member").find_all("ul"): # 会場を列挙する
            place = speech.find("li", class_="place").find("p")
            print(name, date.get_text(), place.get_text())

安倍 晋三 10月10日 ( 火 ) 踊る子馬亭向かい広場 （ 福島市佐原 ）
安倍 晋三 10月10日 ( 火 ) 一関市役所前
安倍 晋三 10月10日 ( 火 ) 仙台駅東口松栄不動産前
安倍 晋三 10月10日 ( 火 ) アエル前
高村 正彦 10月10日 ( 火 ) 山口ﾎﾞｳﾘﾝｸﾞの森駐車場 （ 山口市泉町9-1 ）
高村 正彦 10月10日 ( 火 ) 防府駅みなと口広場 （ 防府市戎町1丁目 ）
高村 正彦 10月10日 ( 火 ) 周南事務所前 （ 周南市川手2-10-10 ）
二階 俊博 10月10日 ( 火 ) 長岡市役所山古志支所前 （ 長岡市山古志竹沢乙461 ）
二階 俊博 10月10日 ( 火 ) 和島　島崎　木村邸 （ 長岡市島崎4688-2 ）
二階 俊博 10月10日 ( 火 ) 栃尾文化センター （ 長岡市中央公園1-35 ）
二階 俊博 10月10日 ( 火 ) ジオワールドビップ （ 三条市旭町2-8-57 ）
竹下 亘 10月10日 ( 火 ) 大平インター付近 （ 浦添市安波茶1-32-2 ）
竹下 亘 10月10日 ( 火 ) 旧よなばる食堂駐車場付近 （ 与那原町与那原513 ）
岸田 文雄 10月10日 ( 火 ) 丸亀ドーム広場（高松壱番館前） （ 高松市丸亀町１－１ ）
岸田 文雄 10月10日 ( 火 ) 瀬戸隆一選挙事務所 （ 坂出市白金町３－８－３５ ）
岸田 文雄 10月10日 ( 火 ) 丸亀市民ひろば（丸亀城の北） （ 丸亀市大手町2-4-1 ）
岸田 文雄 10月10日 ( 火 ) 香川２区自民党事務所 （ さぬき市志度１００８ ）
岸田 文雄 10月10日 ( 火 ) 自民党伊予三島支部 （ 四国中央市中曽根町411-1 ）
岸田 文雄 10月10日 ( 火 ) 妻鳥土地改良区事務所前 （ 四国中央市妻鳥町1121-1 ）
岸田 文雄 10月10日 ( 火 ) 新居浜グランフジ前 （ 新居浜市新須賀町2-10-7 ）
岸田 文雄 10月10日 ( 火 ) 新居浜裁判所前 （ 新居浜市繁本町2-1 ）
橋本 聖子 10月10日 ( 火 ) 堀内　詔子選挙事務所 （ 山梨県富士吉田市中曽根 ）
森山 裕 10月15日 ( 日 ) 綾上農村環境改善センター （ 綾歌郡綾川町山田下3300 ）
森山

ファイルに書き出すなら

In [4]:
# 上記と全く同じだが、結果をファイルにも書き出す。

# 書き出し用ファイルを作る
output = open("itinerary2017.csv","w")

f = open(targetFileName,"r")
bsObj = BeautifulSoup( f.read(), "html")

for person_div in bsObj.find_all("div", class_="speech__accordion"):
    name = person_div.find("div", class_="speech__list--a").find("dl").find("dd").find("strong",class_="name").get_text()
    print( name )
    schedules = person_div.find("div", class_="speech__accordion--inr")
    for date, speeches in zip( schedules.find("h3", class_="speech__list--title"), schedules.find_all("div", class_="speech__table") ):
        # print(speeches)
        for speech in speeches.find("dd",class_="speech__table--member").find_all("ul"):
            place = speech.find("li", class_="place").find("p")
            print(name, date.get_text(), place.get_text())
            output.write( "{0:s},{1:s},{2:s}\n".format(name, date.get_text(), place.get_text()) )
output.close()

安倍 晋三
安倍 晋三 10月10日 ( 火 ) 踊る子馬亭向かい広場 （ 福島市佐原 ）
安倍 晋三 10月10日 ( 火 ) 一関市役所前
安倍 晋三 10月10日 ( 火 ) 仙台駅東口松栄不動産前
安倍 晋三 10月10日 ( 火 ) アエル前
高村 正彦
高村 正彦 10月10日 ( 火 ) 山口ﾎﾞｳﾘﾝｸﾞの森駐車場 （ 山口市泉町9-1 ）
高村 正彦 10月10日 ( 火 ) 防府駅みなと口広場 （ 防府市戎町1丁目 ）
高村 正彦 10月10日 ( 火 ) 周南事務所前 （ 周南市川手2-10-10 ）
二階 俊博
二階 俊博 10月10日 ( 火 ) 長岡市役所山古志支所前 （ 長岡市山古志竹沢乙461 ）
二階 俊博 10月10日 ( 火 ) 和島　島崎　木村邸 （ 長岡市島崎4688-2 ）
二階 俊博 10月10日 ( 火 ) 栃尾文化センター （ 長岡市中央公園1-35 ）
二階 俊博 10月10日 ( 火 ) ジオワールドビップ （ 三条市旭町2-8-57 ）
竹下 亘
竹下 亘 10月10日 ( 火 ) 大平インター付近 （ 浦添市安波茶1-32-2 ）
竹下 亘 10月10日 ( 火 ) 旧よなばる食堂駐車場付近 （ 与那原町与那原513 ）
岸田 文雄
岸田 文雄 10月10日 ( 火 ) 丸亀ドーム広場（高松壱番館前） （ 高松市丸亀町１－１ ）
岸田 文雄 10月10日 ( 火 ) 瀬戸隆一選挙事務所 （ 坂出市白金町３－８－３５ ）
岸田 文雄 10月10日 ( 火 ) 丸亀市民ひろば（丸亀城の北） （ 丸亀市大手町2-4-1 ）
岸田 文雄 10月10日 ( 火 ) 香川２区自民党事務所 （ さぬき市志度１００８ ）
岸田 文雄 10月10日 ( 火 ) 自民党伊予三島支部 （ 四国中央市中曽根町411-1 ）
岸田 文雄 10月10日 ( 火 ) 妻鳥土地改良区事務所前 （ 四国中央市妻鳥町1121-1 ）
岸田 文雄 10月10日 ( 火 ) 新居浜グランフジ前 （ 新居浜市新須賀町2-10-7 ）
岸田 文雄 10月10日 ( 火 ) 新居浜裁判所前 （ 新居浜市繁本町2-1 ）
橋本 聖子
橋本 聖子 10月10日 ( 火 ) 堀内　詔子選挙事務所 （ 山梨県富士吉田市中曽根 ）
森山 裕
森山 裕 10月15日

### デザインが変わればやり直し　2019年版


2019年の参院選では、サイトのデザインが変わり、それに伴って作戦も変わる。

このテーブルに関する以下の分析結果を利用する。

1. speech__wrapクラスのdivタグが人を表す
2. 各行のspeech__wrap--profile--nameクラスのdivが人名を表す
3. 各行のspeech__wrap--scheduleクラスがスケジュールを表す
4. その中のdl要素が、ある日の予定を表す
5. その中のdt要素が日付を表す
6. dd要素が演説会を表す
7. schedule--02クラスのdivタグが被応援者を表す
8. schedule--03クラスのdivタグが場所を表す

In [5]:
targetFileName = "raw/LDP2019.html"

f = open(targetFileName,"r")
bsObj = BeautifulSoup( f.read(), "html")

for person_div in bsObj.find_all("div", class_="speech__wrap"):
    name = person_div.find("div", class_="speech__wrap--profile--name").get_text()
    print( name )
    for day_schedule in person_div.find("div", class_="speech__wrap--schedule").find_all("dl"):
        date = day_schedule.find("dt").get_text()
        candidate = day_schedule.find("div", class_="schedule--02").get_text()
        place = day_schedule.find("div", class_="schedule--03").get_text()
        print(date, candidate, place)

安倍 晋三総裁
7月4日（木曜日） 【福島県】森 まさこ あづま果樹園 （ 福島市飯坂町平野西原1-13 ）
7月5日（金曜日） 【新潟県】つかだ 一郎 村上市役所前 （ 村上市三之町1-1 ）
7月6日（土曜日） 【滋賀県】にのゆ 武史 彦根パリヤ前 （ 彦根市長曽根南町472-2 ）
7月7日（日曜日） 【千葉県】豊田 としろう ＪＲ船橋駅南口
7月8日（月曜日） 【岩手県】平野 たつお さくら野百貨店前 （ 北上市本通り2-2-1 ）
7月9日（火曜日） 【香川県】三宅 しんご 高松三越前 （ 高松市内町7-1 ）
7月10日（水曜日） 【山形県】大沼 みずほ 中通り商店街 （ 酒田市中町1 ）
7月11日（木曜日） 【大分県】いそざき 陽輔 別府公園 （ 別府市大字別府野口原3018-1 ）
7月12日（金曜日） 【三重県】吉川 ゆうみ 伊勢神宮外宮前広場 （ 伊勢市豊川町279 ）
7月13日（土曜日） 【青森県】滝沢 もとめ 新町通りパサージュ広場 （ 青森市新町1-8-5 ）
7月14日（日曜日） 【広島県】河井 あんり 本通り・叶や前 （ 広島市中区本通7－21 ）
7月15日（月曜日） 【北海道】岩本 剛人 清田区役所前 （ 札幌市清田区平岡1条1丁目2－1 ）
7月16日（火曜日） 【新潟県】つかだ 一郎 かに池公園 （ 上越市下門前1663 ）
7月17日（水曜日） 【山形県】大沼 みずほ 山形市役所 （ 山形市旅籠町2－3－25 ）
7月18日（木曜日） 【三重県】吉川 ゆうみ 近鉄四日市駅北口 スターアイランド前 （ 四日市市安島1-1-56 ）
7月19日（金曜日） 【青森県】滝沢 もとめ さくら野百貨店弘前店前 （ 弘前市城東北3-10-1 ）
7月20日（土曜日） 【秋田県】なかいずみ 松司 なかいち （ 秋田市中通1 ）
二階 俊博幹事長
7月4日（木曜日） 【山形県】大沼 みずほ 迎賓館アンジェリーナ （ 天童市鍬ノ町2丁目1-50 ）
7月5日（金曜日） 【宮城県】愛知 治郎 石巻グランドホテル （ 石巻市千石町2-10 ）
7月6日（土曜日） 【山梨県】森屋 ひろし アピオ甲府 （ 中巨摩郡昭和町西条3600 ）
7月7日（日曜日） 【大分県】いそざき 陽輔 大南公民館 （ 大分県大分市中戸次4491−2 ）
7月8日（月曜日

printの代わりにファイルに書き出す

In [6]:
output = open("itinerary2019.csv","w")

f = open(targetFileName, "r")
bsObj = BeautifulSoup( f.read(), "html")

for person_div in bsObj.find_all("div", class_="speech__wrap"):
    name = person_div.find("div", class_="speech__wrap--profile--name").get_text()
    print( name )
    for day_schedule in person_div.find("div", class_="speech__wrap--schedule").find_all("dl"):
        date = day_schedule.find("dt").get_text()
        candidate = day_schedule.find("div", class_="schedule--02").get_text()
        place = day_schedule.find("div", class_="schedule--03").get_text()
        print(date, candidate, place)
        output.write( "{0:s},{1:s},{2:s}\n".format(date, name, place) )

安倍 晋三総裁
7月4日（木曜日） 【福島県】森 まさこ あづま果樹園 （ 福島市飯坂町平野西原1-13 ）
7月5日（金曜日） 【新潟県】つかだ 一郎 村上市役所前 （ 村上市三之町1-1 ）
7月6日（土曜日） 【滋賀県】にのゆ 武史 彦根パリヤ前 （ 彦根市長曽根南町472-2 ）
7月7日（日曜日） 【千葉県】豊田 としろう ＪＲ船橋駅南口
7月8日（月曜日） 【岩手県】平野 たつお さくら野百貨店前 （ 北上市本通り2-2-1 ）
7月9日（火曜日） 【香川県】三宅 しんご 高松三越前 （ 高松市内町7-1 ）
7月10日（水曜日） 【山形県】大沼 みずほ 中通り商店街 （ 酒田市中町1 ）
7月11日（木曜日） 【大分県】いそざき 陽輔 別府公園 （ 別府市大字別府野口原3018-1 ）
7月12日（金曜日） 【三重県】吉川 ゆうみ 伊勢神宮外宮前広場 （ 伊勢市豊川町279 ）
7月13日（土曜日） 【青森県】滝沢 もとめ 新町通りパサージュ広場 （ 青森市新町1-8-5 ）
7月14日（日曜日） 【広島県】河井 あんり 本通り・叶や前 （ 広島市中区本通7－21 ）
7月15日（月曜日） 【北海道】岩本 剛人 清田区役所前 （ 札幌市清田区平岡1条1丁目2－1 ）
7月16日（火曜日） 【新潟県】つかだ 一郎 かに池公園 （ 上越市下門前1663 ）
7月17日（水曜日） 【山形県】大沼 みずほ 山形市役所 （ 山形市旅籠町2－3－25 ）
7月18日（木曜日） 【三重県】吉川 ゆうみ 近鉄四日市駅北口 スターアイランド前 （ 四日市市安島1-1-56 ）
7月19日（金曜日） 【青森県】滝沢 もとめ さくら野百貨店弘前店前 （ 弘前市城東北3-10-1 ）
7月20日（土曜日） 【秋田県】なかいずみ 松司 なかいち （ 秋田市中通1 ）
二階 俊博幹事長
7月4日（木曜日） 【山形県】大沼 みずほ 迎賓館アンジェリーナ （ 天童市鍬ノ町2丁目1-50 ）
7月5日（金曜日） 【宮城県】愛知 治郎 石巻グランドホテル （ 石巻市千石町2-10 ）
7月6日（土曜日） 【山梨県】森屋 ひろし アピオ甲府 （ 中巨摩郡昭和町西条3600 ）
7月7日（日曜日） 【大分県】いそざき 陽輔 大南公民館 （ 大分県大分市中戸次4491−2 ）
7月8日（月曜日